# Дипломная работа
## часть 2

**Цель работы**: создать модель, которая сможет предсказывать товары (в нашем случае топ 3), которые будут наиболее интересными и актуальными для покупки, конкретным покупателем. Данными для модели будет информация о покупках, совершенных им самим или "похожими на него" другими покупателями ранее. Поэтому обучение производится на данных прошлого периода, а проверка качества модели на данных последующих периодов. 

### Библиотеки

In [1]:
import datetime
import numpy as np
import pandas as pd
import os
import seaborn as sns
import pandas_profiling

# all lightfm imports 
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm import cross_validation
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

C:\Users\talic\anaconda3\envs\RecSystem\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


### Функции

In [2]:
def display_data(dataframe, list_columns=None):
    '''
    Развернутая информация по датасету с фильтрацией по списку колонок,
    по умолчнию без фильтрации,
    названия признаков упорядочены по алфавиту
    '''
    if list_columns==None:
        index_ = sorted(dataframe.columns)
    else: 
        index_ = sorted([x for x in set(dataframe.dtypes.index).intersection(set(list_columns))])
    
    df = dataframe[index_]
    d = pd.concat([df.dtypes,df.count() + df.isna().sum(),\
               round((df.isna().sum()/(df.count() + df.isna().sum()))*100,2),df.nunique(),],axis=1)
    d.columns = ['Тип', 'Общ.кол', '% пропусков','Кол-во уник.значений']
    display(d)
    
    return

In [24]:
def features_from_data(df, timestamp):
    '''
    Новые признаки из поля datetime
    
    df - Data Frame
    timestamp - name of column to transform new features
    
    '''
    dat = pd.to_datetime(df[timestamp], unit='ms', origin='unix')
    #df['date0'] = dat
    df['date'] = dat.dt.date
    df['hour'] = dat.dt.hour.astype('int8')
    df['month'] = dat.dt.month.astype('int8')
    df['doy'] = dat.dt.dayofyear.astype('int32')
    #df['year'] = dat.dt.year.astype('int16')
    df['weekofyear'] = dat.dt.isocalendar().week.astype('int8')
    df['dow'] = dat.dt.dayofweek.astype('int8')
    
    
def generate_int_id(dataframe, id_col_name):
    """
    Generate unique integer id for users, questions and answers

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe for Users or Q&A. 
    id_col_name : String 
        New integer id's column name.
        
    Returns
    -------
    Dataframe
        Updated dataframe containing new id column 
    """
    new_dataframe=dataframe.assign(
        int_id_col_name=np.arange(len(dataframe))
        ).reset_index(drop=True)
    return new_dataframe.rename(columns={'int_id_col_name': id_col_name})


def generate_feature_list(dataframe, features_name):
    """
    Generate features list for mapping 

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe for Users or Q&A. 
    features_name : List
        List of feature columns name avaiable in dataframe. 
        
    Returns
    -------
    List of all features for mapping 
    """
    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = features.apply(pd.Series).stack().reset_index(drop=True)
    return features


def calculate_auc_score(lightfm_model, interactions_matrix, 
                        question_features, professional_features): 
    """
    Measure the ROC AUC metric for a model. 
    A perfect score is 1.0.

    Parameters
    ----------
    lightfm_model: LightFM model 
        A fitted lightfm model 
    interactions_matrix : 
        A lightfm interactions matrix 
    question_features, professional_features: 
        Lightfm features 
        
    Returns
    -------
    String containing AUC score 
    """
    score = auc_score( 
        lightfm_model, interactions_matrix, 
        item_features=question_features, 
        user_features=professional_features, 
        num_threads=4).mean()
    return score



def take_features(df_events, ind_visitors, event_name):
    '''
    формирует набор признаков для посетителя сайта по виду события
    (т.е строку из товаров привлекших внимание посетителя при просмотрах, добавлении в корзину или покупке
    понедельно!!)
    '''
    tmp = events0.loc[(events0.visitorid.isin(ind_visitors))&(events0.event==event_name)][['visitorid', 'itemid', 'weekofyear']]
    tmp = tmp.dropna()
    tmp['itemid'] = tmp['itemid'].astype(str)
    #we group all of tags of each user into single row 
    tmp = tmp.groupby(['weekofyear','visitorid'])['itemid'].apply(','.join).reset_index()
    #tmp['itemid'] = ( tmp['itemid'].str.split(',').apply(set).str.join(','))
    tmp.rename(columns={'itemid': 'visitor_'+event_name}, inplace=True)
    
    return df_events.merge(tmp, how='left', on=['visitorid','weekofyear'])

def create_list_parentids2(categoryid):
    '''
    Определяет список всех "предков" в иерархии элемента в обратном порядке
    '''
    list_parentids = categoryid
    
    parent_id = category[category.categoryid==int(categoryid)].parentid.values[0]
    
    while not parent_id.astype(str)=='-1':
        list_parentids = ','.join([list_parentids, str(int(parent_id))])
        category_id = parent_id
        
        parent_id = category[category.categoryid==category_id].parentid.values[0]
            
    return list_parentids


def create_df_feature2(code_property):
    '''
    Создает датафрейм для добавления к данным по коду свойства
    '''
    
    new_df = properties[properties['property']==code_property][['itemid','weekofyear', 'value']]
    
    if code_property == '790': # price
        new_df['value'] = new_df['value'].apply(lambda x: x[1:])
        new_df['value'] = new_df['value'].astype('float')
       
    else:    
        if code_property == 'categoryid': 
            new_df['value'] = new_df['value'].apply(lambda x: create_list_parentids2(x))
            
        new_df['value'] = new_df['value'].str.split(' ').apply(set).str.join(',')

    # Свойства устанавливаются на следующую неделю:
    new_df['weekofyear'] = new_df['weekofyear'] + 1 

    return new_df

def create_features_visitors(df_visitors, df_events, event_name):
    '''
    формирует набор признаков для посетителя сайта по виду события
    (т.е строку из товаров привлекших внимание посетителя при просмотрах, добавлении в корзину или покупке
    )
    '''
    tmp = df_events[['visitorid', event_name]]
    tmp = tmp.dropna()
    
    #we group all of tags of each user into single row 
    tmp = tmp.groupby(['visitorid'])[event_name].apply(','.join).reset_index()
    tmp[event_name] = ( tmp[event_name].str.split(',').apply(set).str.join(','))
        
    return df_visitors.merge(tmp, how='left', on=['visitorid'])

def create_features(dataframe, features_name, id_col_name):
    """
    Создаются функции, которые будут готовы для загрузки в light fm

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe со свойствами
    features_name : List
        Список имен столбцов объектов, доступных во фрейме данных
    id_col_name: String
        Имя столбца, содержащее идентификатор, с которым будут сопоставлены объекты.
        1. items_id_num
        2. visitors_id_num

    Returns
    -------
    Pandas Series
        Серия pandas, содержащая технологические функции, готовые для подачи в light fm.
    """

    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = list(zip(dataframe[id_col_name], features))
    return features

In [4]:
'''
Объявление переменных

'''
path = r'F:\SkillFactory\diplom'
dict_actions = {0:'view', 1:'addtocart', 2: 'transaction'}


### Загрузка исходгого датасета

In [187]:
'''
Загрузка данных

'''

category = pd.read_csv(os.path.join(path,'category_tree.csv'))
events = pd.read_csv(os.path.join(path,'events.csv'))
items1 = pd.read_csv(os.path.join(path,'item_properties_part1.csv'))
items2 = pd.read_csv(os.path.join(path,'item_properties_part2.csv'))

In [188]:
'''
Получение данных 

'''
# Заполняем -1 корень дерева (25 штук)
category['parentid'] = category['parentid'].fillna(-1)
    
# Уменьшаем размерность данных
category['categoryid'] = category['categoryid'].astype('int32')
category['parentid'] = category['parentid'].astype('int32')

# Получим признаки из даты
features_from_data(events, 'timestamp')

#Дубликаты
events = events.drop_duplicates()

#for col in ['visitorid', 'itemid', 'transactionid']:
#    events[col] = events[col].astype('int32')
    
# Сохранение
events0 = events.copy() 

#Объединяем свойства в один датасет:
properties = pd.concat([items1,items2])
properties = properties.drop_duplicates()

# Уменьшаем размерность данных
properties['itemid'] = properties['itemid'].astype('int32')

### Описание исходных данных

In [7]:
category.head(3)

,categoryid,parentid
0,1016,213
1,809,169
2,570,9


In [8]:
display_data(category)

,Тип,Общ.кол,% пропусков,Кол-во уник.значений
categoryid,int32,1669,0.0,1669
parentid,int32,1669,0.0,363


Датафрейм **category** содержит дерево категорий для товаров

In [9]:
properties.head(3)

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566


In [10]:
display_data(properties)

,Тип,Общ.кол,% пропусков,Кол-во уник.значений
itemid,int32,20275902,0.0,417053
property,object,20275902,0.0,1104
timestamp,int64,20275902,0.0,18
value,object,20275902,0.0,1966868


Датафрейм **properties** содержит информацию о свойстве('**property**') товара ('**itemid**'), имеющем значение ('**value**') на определенную дату ('**date**'). Как показал анализ установка значений свойств происходит 1 раз в неделю (см. часть 1) 
Поле **date** позволяет создать дополнительные свойства: **dow**, **doy**, **hour**, **month**, **weekofyear**.

In [11]:
events.head(3)

,timestamp,visitorid,event,itemid,transactionid,date,hour,month,doy,weekofyear,dow
0,1433221332117,257597,view,355908,NaN,2015-06-02,5,6,153,23,1
1,1433224214164,992329,view,248676,NaN,2015-06-02,5,6,153,23,1
2,1433221999827,111016,view,318965,NaN,2015-06-02,5,6,153,23,1


In [15]:
display_data(events)

,Тип,Общ.кол,% пропусков,Кол-во уник.значений
date,object,2755641,0.00,139
dow,int8,2755641,0.00,7
doy,int32,2755641,0.00,139
event,object,2755641,0.00,3
hour,int8,2755641,0.00,24
itemid,int64,2755641,0.00,235061
month,int8,2755641,0.00,5
timestamp,int64,2755641,0.00,2750455
transactionid,float64,2755641,99.19,17672
visitorid,int64,2755641,0.00,1407580


Датафрейм **events** содержит информацию о событии ('**event**') для покупателя ('**itemid**') в определенное время ('**timestamp**'). Если это покупка ('**event=transaction**'), то имеется № транзакции ('**transactionid**'). Как и в предыдущем датасете можно получить дополнительные свойства **dow**, **doy**, **hour**, **month**, **weekofyear**, а также информацию о количестве покупок  в данном чеке ('**transaction_q**')

### Трансформации исходного датасета, чтобы получить датасет для обучения

In [189]:
'''
Cоздание новых признаков

'''

# для избранных покупателей построить списки по просмотрам, корзинам и покупкам
#Для каждого чека transactionid посчитаем количество покупок в чеке - transactionid
tmp = pd.DataFrame(events.groupby('transactionid').transactionid.count())
tmp.rename(columns={'transactionid': 'transaction_q'}, inplace=True)
tmp.reset_index(inplace=True)
events = events.merge(tmp,left_on='transactionid', right_on='transactionid' )


In [190]:
'''    
Сформируем датафрейм с покупаемыми товарами и только с покупателями
'''

ind = events[events.transaction_q>2].itemid.tolist() # список покупаемых товаров
ind = list(set(ind))

ind_visitors = events[events.transaction_q>2].visitorid.tolist() # список покупателей
ind_visitors = list(set(ind_visitors))

df = events.loc[events.itemid.isin(ind)].copy()
df = df.loc[df.visitorid.isin(ind_visitors)].copy()  

#Сформируем датафрейм с покупаемыми товарами
properties = properties.loc[properties.itemid.isin(ind)].copy()


df_events = df.copy()
df_events = df_events[['visitorid','itemid', 'transactionid','hour','month','doy','weekofyear','dow','transaction_q']]

# Число покупок за неделю у покупателя
# Число продаж за неделю данного товара
df_events['buying'] = df_events.groupby(['weekofyear', 'visitorid'])['transaction_q'].\
    transform('count').astype('int32')
df_events['sales'] = df_events.groupby(['weekofyear', 'itemid'])['transaction_q'].\
    transform('count').astype('int32')
    
#Признаки по виду события
for event_name in ['view', 'addtocart', 'transaction']:
    df_events = take_features(df_events, ind_visitors, event_name)
    
# Сформируем список наиболее важных наиболее распространенных свойств
list_properties = properties.drop_duplicates(['itemid', 'property']).groupby("property")['itemid'].count().sort_values(ascending=False)[:25].index.tolist()

# Получим признаки из даты
features_from_data(properties, 'timestamp')
properties = properties.drop(columns='timestamp')

# Отбор покупаемых элементов
df_items = df[(df.transactionid>-1)][['transactionid','visitorid','itemid', 'weekofyear',]].copy()

for col in ['visitorid', 'itemid', 'transactionid']:
    df_items[col] = df_items[col].astype('int32')


Как оценить товар, который предпочитает визитер? Наверное, по количеству приобретаемого товара.
Все было бы хорошо, но в данном датасете, как показывает анализ, основная часть покупок - это товары,
которые приобретаются не более одного раза. Однако пусть будет так:

**rank** - целевая функция, позволяет оценить количество купленного товара

In [191]:
# Количество продаж по товару за период: неделя
# rank - целевая функция, позволяет оценить количество купленного товара

dict_check = pd.DataFrame(df_items.groupby(['itemid','weekofyear'])['transactionid'].count())\
    .reset_index()
dict_check = dict_check.rename(columns={'transactionid': 'rank'})
df_items = df_items.merge(dict_check, on=['itemid','weekofyear'], how='left')
df_items['rank'] = df_items['rank'].astype('int16')


df_items = pd.DataFrame(df_items.groupby(['itemid', 'weekofyear'])['rank'].sum())\
    .reset_index()
df_items = df_items.drop_duplicates()    

Среди свойств в датафрейме **properties** имеется свойство с **categoryid** = 790 числового типа с достаточно умеренным количеством пропусков и по частоте встречаемости, занимающее 2 -ое место среди всех признаков. Есть предположение, что это ничто иное как цена товара. Поэтому выделим этот признак в отдельный столбец, а затем используем для формирования новых признаков. 

In [192]:
# Присоединим цену
new_df = create_df_feature2('790') #price
new_df['value'].fillna(1,inplace=True)
df_items = pd.merge(df_items, new_df, left_on=['itemid', 'weekofyear'],right_on=['itemid', 'weekofyear'], how='left')
df_items.rename(columns = {'value': 'price'},  inplace=True)

In [193]:
df_items.head(3)

,itemid,weekofyear,rank,price
0,15,28,1,8400.0
1,19,33,1,18600.0
2,25,24,1,37320.0


In [200]:
df_items[df_items.price>0].price.agg(['min', 'max'])

min          1.0
max    2246640.0
Name: price, dtype: float64

In [199]:
# Пустые значения и 0 заменим на 1
df_items['price'].fillna(1, inplace=True)
df_items['price'] = np.where(df_items['price']==0, 1, df_items['price'])

Теперь поле **rank** можно откорректировать так, чтобы оно имело не количественный смысл, а денежный. Тогда мы будем ранжировать товар по количеству прибыли, что собственно и требуется в постановке задачи:

In [202]:
df_items['rank'] = df_items['rank']*df_items['price']

In [204]:
'''
Поскольку свойства товара изменялись 1 раз в неделю, для каждого itemid
определяем значение week_item_tags - набор свойств товара на этой неделе,
кроме цены, которая вынесена отдельно

'''
df_items['week_item_tags'] = ''

for i in range(len(list_properties)):
    code_property = list_properties[i]
    #print(code_property)
    if not(code_property=='790'):
        new_df = create_df_feature2(code_property)
        df_items = pd.merge(df_items, new_df, on=['itemid', 'weekofyear'], how='left')
        df_items['week_item_tags'] = (df_items[['week_item_tags', 'value']].apply(lambda x: ','.join(x.dropna()), axis=1))
        df_items.drop(columns = ['value'], axis=1, inplace=True)
        
df_items['week_item_tags'] = df_items['week_item_tags']\
    .apply(lambda x : x[1:] if x.startswith(",") else x)

In [209]:
df_items[~(df_items['rank']==df_items['price'])]

,itemid,weekofyear,rank,price,week_item_tags
14,1510,29,107520.0,26880.0,"727528,20836,150169,624616,312886,99315,86628,1"
17,1684,31,991200.0,247800.0,1
35,4067,37,771120.0,85680.0,"1137917,3169,931404,1"
38,4887,23,53760.0,13440.0,"679677,1297729,588415,311524,665587,1131293,18..."
54,7804,36,19152.0,4788.0,
...,...,...,...,...,...
4033,461686,33,74400.0,18600.0,"245814,171308,370498,237874"
4037,461686,37,695520.0,19320.0,"245814,171308,370498,237874"
4039,463002,29,98400.0,24600.0,1
4051,464731,27,402408.0,44712.0,"1219,121,540,140,1285872,10317,n60.000,807959,..."


In [210]:
# Тэги на товар строим на основании уникальной нумерации

#item_tags; кто интересовался товаром из визитеров 
item_tags = df_events[['itemid','visitorid']]
item_tags = item_tags.drop_duplicates()
item_tags['visitorid'] = item_tags['visitorid'].astype(str)

item_tags = item_tags.groupby(['itemid'])['visitorid']\
    .apply(','.join).reset_index()

item_tags.rename({'visitorid':'item_tags'}, axis=1, inplace=True)

#генерим уникальный номер
item_tags = generate_int_id(item_tags, 'items_id_num')

In [211]:
# присоединим новые свойства к датафрейму с товарами и событиями
df_items = df_items.merge(item_tags, how='left', on='itemid')
df_events = df_events.merge(df_items[['itemid','weekofyear','items_id_num','item_tags',\
    'price','rank', 'week_item_tags']]\
    , how='left', on=['itemid','weekofyear'])

In [212]:
#visitor_tags: какими свойствами товаров интересовался визитер
visitor_tags = df_events[['visitorid','week_item_tags']]
visitor_tags = visitor_tags.drop_duplicates()

visitor_tags = visitor_tags.groupby(['visitorid'])['week_item_tags']\
    .apply(','.join).reset_index()

In [213]:
visitor_tags.rename({'week_item_tags':'visitor_tags'}, axis=1, inplace=True)
visitor_tags['visitor_tags'] = (visitor_tags['visitor_tags']\
    .str.split(',').apply(set).str.join(','))
visitor_tags['visitor_tags'] = visitor_tags['visitor_tags']\
    .apply(lambda x : x[1:] if x.startswith(",") else x)

In [214]:
# Датафрейс с визитерами и их свойствами    
df_visitors = df_events.groupby(['visitorid'])[['transactionid']]\
    .count().reset_index()
df_visitors = df_visitors.merge(visitor_tags, how='left', on='visitorid').fillna('')

#генерим уникальный номер
df_visitors = generate_int_id(df_visitors, 'visitors_id_num')

# присоединим новые свойства к датафрейму событиями
df_events = (df_events.merge(df_visitors[['visitorid', 'visitors_id_num', 'visitor_tags']]\
    ,how='inner', on='visitorid'))

In [98]:
 # ну и какие у нас фичи получились?
cols = df_events.columns

In [215]:
 for col in ['visitor_view', 'visitor_addtocart', 'visitor_transaction']:
        
    # заполним пропуски        
    df_events[col].fillna('', inplace=True)    
    
    df_visitors = create_features_visitors(df_visitors, df_events, col)

In [216]:
df_visitors

,visitorid,transactionid,visitor_tags,visitors_id_num,visitor_view,visitor_addtocart,visitor_transaction
0,3465,3,"1154859,250613,1,1083967,17996,1318567,1285872",0,"114485,8523","114485,8523,434048","114485,8523,434048"
1,4101,4,"1,963713,645524,n210852.000,519769",1,"400859,115244,376365,401523,204798,457408,3566...","228066,400859,170353,104752","228066,400859,170353,104752"
2,6468,4,"1135780,n9216.000,842796,221748,n16392.000,128...",2,"65273,108343,277247,278166,378760,289096","289096,378760,108343,65273","378760,108343,65273,289096"
3,6952,3,"350726,1,171308,207130,n36.000,1297729,370498,...",3,"171878,108924,218794,461686,419052","108924,19789,461686","108924,19789,461686"
4,6958,4,"1330686,1",4,"371281,236652,166378,431417,374855,232462,1872...","339822,187200,184011,431417,374855","339822,431417,184011,187200"
...,...,...,...,...,...,...,...
708,1391065,3,,708,"88160,335033,80885,48115","88160,335033,48115","88160,335033,48115"
709,1392423,3,"1154859,1,1250749,961511,653725,424314",709,"445351,37029,240721,308551,213834","445351,37029","445351,37029,213834"
710,1398978,9,"1146838,150169,n24000.000,1290536,655992,92743...",710,127080,"154775,8015,62805,31952,49405,336668,38083,252...","8015,62805,49405,336668,38083,252023,330029,87..."
711,1400296,3,"150169,89463,834659,858078,679677,312,1128903,...",711,"369902,101874,249477","369902,101874,249477","369902,101874,249477"


### Краткое описание модели

Для построение модели будем использовать гибридную модель LigthFM, как широко распространенный пакет "из коробки" для построения рекомендаций для предоставления рекомендаций на данных, где пользователи взаимодействуют с подмножеством элементов из конечного набора:

**predict(0, item_ids=[0, 1, 2], item_features=[11's features, 12's features, 13's features], user_features=[D.features])**

Модель изучает вложения (скрытые представления в многомерном пространстве) для пользователей и элементов таким образом, чтобы кодировать предпочтения пользователя по отношению к элементам. При умножении вместе эти представления дают баллы по каждому элементу для данного пользователя; элементы, получившие высокие оценки, с большей вероятностью будут интересны пользователю.

Представления пользователя и элемента выражаются в терминах представлений их функций: для каждой функции оценивается эмбеддинг, и затем эти функции суммируются, чтобы получить представления для пользователей и элементов.

Эмбеддинги изучаются с помощью методов стохастического градиентного спуска.

Доступны четыре функции потери:

- логистическая: используется, когда присутствуют как положительные (1), так и отрицательные (-1) взаимодействия.

- BPR: Максимизирует разницу в прогнозе между положительным примером и случайно выбранным отрицательным примером. Используется, когда присутствуют только положительные взаимодействия и желательно оптимизировать ROC AUC.

- WARP: Максимизирует ранг положительных примеров путем многократной выборки отрицательных примеров до тех пор, пока не будет найден один, нарушающий ранг. Используется, когда присутствуют только положительные взаимодействия и желательно оптимизировать верхнюю часть списка рекомендаций (точность @ k).

- k-OS WARP: потеря статистики k-го порядка 3. Модификация WARP, которая использует k-й положительный пример для любого данного пользователя в качестве основы для попарных обновлений.

- Доступны два графика скорости обучения: adagrad: 4, adadelta: 5

#### Гиперпараметры: 
- no_components (int, необязательно) – размерность скрытых вложений объектов.

- k (int, необязательно) – для обучения k-OS k-й положительный пример будет выбран из n положительных примеров, отобранных для каждого пользователя.

- n (int, необязательно) – для обучения k-OS максимальное количество положительных результатов, отобранных для каждого обновления.

- learning_schedule (строка, необязательно) – один из (‘adagrad’, ‘adadelta’).

- loss (строка, необязательно) – одна из (‘logistic’, ‘bpr’, ‘warp’, ‘warp-kos’): функция потерь.

- learning_rate (float, необязательно) – начальная скорость обучения для расписания обучения adagrad.

- rho (float, необязательно) – коэффициент скользящей средней для расписания обучения adadelta.

- epsilon (float, необязательно) – параметр настройки для расписания обучения adadelta.

- item_alpha (float, необязательно) – штраф L2 за элементы элемента. Совет: установка этого значения слишком высоко может замедлить тренировку. Один из хороших способов проверить - оказались ли конечные веса во вложениях в основном равными нулю. Та же идея применима и к параметру user_alpha.

- user_alpha (float, необязательно) – штраф L2 за пользовательские функции.

- max_sampled (int, необязательно) – максимальное количество отрицательных выборок, используемых во время подгонки основы. Требуется много выборок, чтобы найти отрицательные триплеты для пользователей, которые уже хорошо представлены моделью; это может привести к очень длительному времени обучения и переобучению. Установка этого значения на большее число обычно приводит к увеличению времени обучения, но в некоторых случаях может повысить точность.

- random_state (int seed, RandomState instance или None) – начальное значение генератора псевдослучайных чисел, используемое при перетасовке данных и инициализации параметров.


#### Переменные:
- ~LightFM.item_embeddings (массив np.float32 формы [n_item_features, n_components]) – Содержит оцененные скрытые векторы для характеристик элемента. [i, j]-я запись дает значение j-го компонента для i-го элемента элемента. В простейшем случае, когда матрица признаков элемента является идентификационной матрицей, i-я строка будет представлять скрытый вектор i-го элемента.

- ~LightFM.user_embeddings (массив np.float32 формы [n_user_features, n_components]) – Содержит оценочные скрытые векторы для пользовательских функций. [i, j]-я запись дает значение j-го компонента для i-й пользовательской функции. В простейшем случае, когда матрица пользовательских признаков является идентификационной матрицей, i-я строка будет представлять i-й скрытый вектор пользователя.

- ~LightFM.item_biases (массив np.float32 формы [n_item_features,]) – содержит смещения для item_features.

- ~LightFM.user_biases (массив np.float32 формы [n_user_features,]) – содержит смещения для user_features.

### Формат входных данных для  модели LightFM

Поскольку модель LightFM требует определенного формата входных данных, исходные данные потребуют дополнительных преобразований. Прежде всего выделим отдельно каждое свойство для товаров и визитеров

In [217]:
# сгенерируем списки свойств для товаров и для визитеров

item_feature_list = generate_feature_list(
    item_tags,
    ['item_tags'])

visitor_feature_list = generate_feature_list(
    df_visitors,
    ['visitor_tags'])

Теперь необходимо создать функции для подачи в LightFM: каждому идентификатору товара или визитера ставится в соответствие вектор с его свойствами

In [220]:
list(cols)

['buying',
 'week_item_tags',
 'rank',
 'sales',
 'visitor_view',
 'item_tags',
 'transaction_q',
 'weekofyear',
 'visitor_tags',
 'visitor_transaction',
 'visitors_id_num',
 'price',
 'items_id_num',
 'visitor_addtocart']

In [219]:
cols = set(cols)-{'visitorid', 'itemid','doy', 'dow', 'month', 'hour', 'transactionid'}

In [253]:
# создание функций для подачи в lightfm 

df_events2 = df_events[cols]
#df_events2['total_weights'] = 1/(df_events2['rank']) 
df_events2['total_weights'] = (df_events2['rank']) 

item_tags['item_features'] = create_features(
    item_tags, ['item_tags'], 
    'items_id_num')

df_visitors['visitor_features'] = create_features(
    df_visitors,
    ['visitor_tags'],
    'visitors_id_num')

C:\Users\talic\AppData\Local\Temp\ipykernel_1516\2522923461.py:3: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_events2 = df_events[cols]
C:\Users\talic\AppData\Local\Temp\ipykernel_1516\2522923461.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_events2['total_weights'] = (df_events2['rank'])


In [254]:
item_tags.items_id_num.nunique(), df_visitors.visitors_id_num.nunique()

(2952, 713)

Теперь можно построить датасет в формате LightFM, подав на вход датасета новые уникальные номера объектов и полученные функции

In [260]:
########################
# Dataset building for lightfm
########################

dataset = Dataset()
dataset.fit(
    set(df_visitors['visitors_id_num']), 
    set(item_tags['items_id_num']),
    item_features=item_feature_list, 
    user_features=visitor_feature_list)

In [261]:
df_train = df_events2[df_events2['weekofyear']<35].copy()
df_test = df_events2[df_events2['weekofyear']>34].copy()

Теперь мы строим матрицу взаимодействий между посетителями и элементами, мы передаем идентификаторы посетителей и элементов в виде кортежа, например, pd.Series((visitor_id, item_id), (visitor_id, item_id)) затем мы используем метод light fm build in для построения матрицы взаимодействий

In [262]:
df_events2['visitor_id_tuple'] = list(zip(
    df_events2.visitors_id_num,\
    df_events2.items_id_num,\
    df_events2.total_weights))

interactions, weights = dataset.build_interactions(df_events2['visitor_id_tuple'])

C:\Users\talic\AppData\Local\Temp\ipykernel_1516\1763872109.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_events2['visitor_id_tuple'] = list(zip(


In [263]:
df_train['visitor_id_tuple'] = list(zip(
    df_train.visitors_id_num,\
    df_train.items_id_num,\
    df_train.total_weights))

interactions_train, weights_train = dataset.build_interactions(df_train['visitor_id_tuple'])

In [264]:
df_test['visitor_id_tuple'] = list(zip(
    df_test.visitors_id_num,\
    df_test.items_id_num,\
    df_test.total_weights))

interactions_test, weights_test = dataset.build_interactions(df_test['visitor_id_tuple'])

Теперь мы создаем наши функции таким образом, чтобы их понимал lightfm, для этого
используем метод light fm build

In [265]:
items_features = dataset.build_item_features(
    item_tags['item_features'])

visitor_features = dataset.build_user_features(
    df_visitors['visitor_features'])

### Построение модели

Определяем модель lightfm, указав гиперпараметры.
Затем обучаем модель на матрице взаимодействий, элементам и пользовательским функциям

In [229]:
# Построение модели

model = LightFM(
    learning_rate=0.05,
    no_components=300,
    loss='warp',
    random_state=42)

model.fit(
    interactions_train,
    item_features=items_features,
    user_features=visitor_features, sample_weight=weights_train,
    epochs=5, num_threads=4, verbose=True)

Epoch: 100%|██████████| 5/5 [00:13<00:00,  2.67s/it]


In [230]:
# для 20 - 0.4964845
# для всех - 0.93770784
# 0.9575064 с ценой
# 0.96793693
# 0.93443596
calculate_auc_score(model, interactions_train, items_features, visitor_features)

0.770502

In [231]:
calculate_auc_score(model, interactions_test, items_features, visitor_features)

0.39934668

In [232]:
# Построение модели

model = LightFM(
    no_components=300,
    learning_rate=0.05,
    loss='warp',
    learning_schedule='adadelta',
    random_state=42)

model.fit(
    interactions_train,
    item_features=items_features,
    user_features=visitor_features, sample_weight=weights_train,
    epochs=5, num_threads=4, verbose=True)

Epoch: 100%|██████████| 5/5 [00:19<00:00,  3.94s/it]


In [233]:
calculate_auc_score(model, interactions_train, items_features, visitor_features)

0.9686787

In [234]:
calculate_auc_score(model, interactions_test, items_features, visitor_features)

0.65387315

In [235]:
patks = precision_at_k(model, interactions_train,
    train_interactions=interactions_train,
    item_features=items_features,   
    user_features=visitor_features,
    k=3, num_threads=4, check_intersections=False)

print('Hybrid train set p@3: %s' % np.mean(patks))

Hybrid train set p@3: 0.7527413


In [236]:
patks = precision_at_k(model, interactions_test,
    train_interactions=interactions_train,
    item_features=items_features,   
    user_features=visitor_features,
    k=3, num_threads=3, check_intersections=False)

print('Hybrid train set p@3: %s' % np.mean(patks))

Hybrid train set p@3: 0.08201058


In [237]:
# Для best model:


#epochs, learning_rate,\
#no_components, alpha = [248, # epochs
#    0.581181804647835,  # learning_rate
#    117, # no_components
#    0.00022597769528813776, # alpha
#    ]

#epochs, learning_rate,\
#no_components, alpha = [88, #epochs
#    0.3943014513515949, #learning_rate:
#    95, #no_components:
#    3.356245782309117e-05, #alpha: 
#    ]
epochs, learning_rate,\
no_components, alpha = [150,
    0.05,
    300,
    0.0,
    ]
    
    
user_alpha = alpha
item_alpha = alpha

model = LightFM(
no_components=no_components,
learning_rate=learning_rate,
loss='warp',
learning_schedule='adadelta',    
random_state=42,
user_alpha=user_alpha,
item_alpha=item_alpha)
print('epochs='.upper(), epochs)
model.fit(
    interactions_train,
    item_features=items_features,
    user_features=visitor_features, sample_weight=weights_train,
    epochs=epochs, num_threads=4, verbose=True)    

EPOCHS= 150


Epoch: 100%|██████████| 150/150 [00:48<00:00,  3.06it/s]


In [238]:
calculate_auc_score(model, interactions_train, items_features, visitor_features)

0.99997973

In [239]:
calculate_auc_score(model, interactions_test, items_features, visitor_features)

0.70773005

In [240]:
patks = precision_at_k(model, interactions_train,
    train_interactions=interactions_train,
    item_features=items_features,   
    user_features=visitor_features,
    k=3,  check_intersections=False)

print('Hybrid train set p@3: %s' % len(patks), np.mean(patks))

Hybrid train set p@3: 608 1.9698466


In [241]:
patks = precision_at_k(model, interactions_test,
    train_interactions=interactions_train,
    item_features=items_features,   
    user_features=visitor_features,
    k=3,  check_intersections=False)

print('Hybrid train set p@3: %s' % len(patks),np.mean(patks))

Hybrid train set p@3: 126 0.1904762


## Оптимизация гиперпараметров с scikit-optimize

**Внимание!** Для оптимизации гиперпараметров была испробована библиотека **skopt** 
Однако не все версии python поддерживают ее!

In [268]:
def objective(params):
    # unpack
    epochs, learning_rate,\
    no_components, alpha = params
    
    user_alpha = alpha
    item_alpha = alpha
    
    model = LightFM(
    no_components=no_components,
    learning_rate=learning_rate,
    loss='warp',
    random_state=42,
    user_alpha=user_alpha,
    item_alpha=item_alpha)
    print('epochs='.upper(), epochs)
    model.fit(
        interactions_train,
        item_features=items_features,
        user_features=visitor_features, sample_weight=weights_train,
        epochs=epochs, num_threads=4, verbose=True)
    
    
    patks = precision_at_k(model, interactions_test,
      train_interactions=interactions_train,
      item_features=items_features,   
      user_features=visitor_features,
      k=3, num_threads=4, check_intersections=False)
    mapatk = np.mean(patks)
    # Make negative because we want to _minimize_ objective
    out = -mapatk
    # Handle some weird numerical shit going on
    if np.abs(out + 1) < 0.01 or out < -1.0:
        return 0.0
    else:
        return out

In [ ]:
space = [(1, 200), # epochs
    (10**-4, 1.0, 'log-uniform'), # learning_rate
    (20, 200), # no_components
    (10**-6, 10**-1, 'log-uniform'), # alpha
    ]
res_fm = forest_minimize(objective, space, n_calls=100, random_state=0,verbose=True)


## Кросс-валидация на лучшей модели

In [266]:
sorted(df_events2['weekofyear'].unique())

[18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38]

In [242]:
def best_model(params):
    # unpack
    epochs, learning_rate,\
    no_components, alpha = params
    
    user_alpha = alpha
    item_alpha = alpha
    
    train['visitor_id_tuple'] = list(zip(
    train.visitors_id_num,\
    train.items_id_num,\
    train.total_weights))

    interactions_train, weights_train = dataset.build_interactions(train['visitor_id_tuple'])

    test['visitor_id_tuple'] = list(zip(
    test.visitors_id_num,\
    test.items_id_num,\
    test.total_weights))

    interactions_test, weights_test = dataset.build_interactions(test['visitor_id_tuple'])
    
    model = LightFM(
    no_components=no_components,
    learning_rate=learning_rate,
    learning_schedule='adadelta',        
    loss='warp',
    random_state=42,
    user_alpha=user_alpha,
    item_alpha=item_alpha)
        
    model.fit(
        interactions_train,
        item_features=items_features,
        user_features=visitor_features, sample_weight=weights_train,
        epochs=epochs, num_threads=4, verbose=True)
    
    
    patks = precision_at_k(model, interactions_train,
      train_interactions=interactions_train,
      item_features=items_features,   
      user_features=visitor_features,
      k=3, check_intersections=False)
    
    print('Hybrid train set p@3: %s' % np.mean(patks))
    
    patks = precision_at_k(model, interactions_test,
      train_interactions=interactions_test,
      item_features=items_features,   
      user_features=visitor_features,
      k=3, check_intersections=False)
    
    print('Hybrid test set p@3: %s' % np.mean(patks))
    
    # Make negative because we want to _minimize_ objective
    #out = -mapatk
    # Handle some weird numerical shit going on
    '''
    if np.abs(out + 1) < 0.01 or out < -1.0:
        return 0.0
    else:
        return out
    '''
    

In [243]:
week_index = sorted(df_events2['weekofyear'].unique())
week_index[8]

26

In [270]:
'''
Maximimum p@k found: 0.32419
Optimal parameters:
epochs: 248
learning_rate: 0.581181804647835
no_components: 117
alpha: 0.00022597769528813776
'''
'''
params = [248, # epochs
    0.581181804647835,  # learning_rate
    117, # no_components
    0.00022597769528813776, # alpha
    ]
'''    
params = [100,
    0.05,
    300,
    0.0,
    ]

for _index in range(15, len(week_index)):
    print(_index)
    train = df_events2[df_events2['weekofyear']<_index].copy()
    test = df_events2[df_events2['weekofyear']==_index].copy()
    best_model(params)

15


Epoch: 100%|██████████| 100/100 [00:00<00:00, 204.52it/s]
C:\Users\talic\AppData\Roaming\Python\Python39\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\talic\AppData\Roaming\Python\Python39\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


Hybrid train set p@3: nan
Hybrid test set p@3: nan
16


Epoch: 100%|██████████| 100/100 [00:00<00:00, 131.57it/s]


Hybrid train set p@3: nan
Hybrid test set p@3: nan
17


...

Hybrid train set p@3: nan
Hybrid test set p@3: 0.0
19


Epoch: 100%|██████████| 100/100 [00:00<00:00, 133.69it/s]


Hybrid train set p@3: 0.88888884
Hybrid test set p@3: 0.008333334
20


Epoch: 100%|██████████| 100/100 [00:04<00:00, 24.34it/s]


Hybrid train set p@3: 1.2301588
Hybrid test set p@3: 0.013333334


### Команды для Докер_образа

FROM ubuntu:latest

RUN apt-get update \
    && apt-get install -qyy -o APT::Install-Recommends=false -o APT::Install-Suggests=false \
    file \
    gcc \
    python3 \
    python3-dev \
    python3-pip \
    python3-setuptools \
    python3-venv \
    python3-wheel \
    && apt-get clean \
    && rm -rf /var/lib/apt/lists/*

RUN pip3 install --cache-dir=/tmp/pipcache --upgrade pip && rm -rf /tmp/pipcache
RUN pip install --cache-dir=/tmp/pipcache poetry && rm -rf /tmp/pipcache


FROM python:latest

WORKDIR /src/app

COPY requirements.txt ./

RUN pip install --no-cache-dir -r requirements.txt

COPY /src/ .

CMD ["python", "./main.py"]

## Выводы

1. Полученный результат на метрике Precision@3 = 0.19 может быть улучшен введением дополнительных функций для как для визитера, так и для товара, полученных на основании временных признаков: дня недели, часа, месяца покупки и прочих, сочетания ценовых признаков, а также с использованием комбинированного подхода к учету просмотра, использования корзины и покупки и подбора соответствующих весов. Далее, поскольку данные носят исторический характер, то влияние функций на результат также нужно осуществлять с подбором соответствующих весов: чем старше данные, тем меньше их влияние на результат.

2. Использование в production:
Для реализации задачи я бы использовала 2 компьютера:
1 послабее с API для пользователя выдавал результат на основании построенной модели для известных пользователей.
Справочники с пользователями, товарами, а также событиями я преобразовала бы в формат Postgresql для быстроты и простоты доступа.
Для переобучения модели на новых данных требуется компьтер помощнее: он подхватывает новые данные о пользователе и его предпочтениях из базы Postgresql и старую модель заменяет на переобученную модель,  также отслеживает актульность модели и точность выдаваемых метрик

